In [45]:
import joblib
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [18]:
with open("random_forest.joblib", "rb") as f:
    random_forest = joblib.load(f)

C:\Users\Guillem\Anaconda3\envs\ucode\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Guillem\Anaconda3\envs\ucode\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [29]:
DATA_DIR = Path("../../data/public/football-data/data")

In [41]:
df = pd.DataFrame()
subjects = ["Subject-000", "Subject-001", "Subject-002", 
            "Subject-003", "Subject-004", "Subject-005",
            "Subject-006", "Subject-007", "Subject-008"]
for s in subjects:
    for d in DATA_DIR.joinpath(s).iterdir():
        current_df = pd.read_csv(str(d))
        y = d.stem.split("-")[0]
        current_df["y"] = y
        if y == "Pass" and np.random.random() > .7:
            df = df.append(current_df[::5])
        elif y == "Dribbling" and np.random.random() > .3:
            df = df.append(current_df[::5])
        elif y == "Walking" or y == "Running":
            current_df["y"] = "Around"
            df = df.append(current_df[::5])
        elif y != "Pass" and y != "Dribbling":
            df = df.append(current_df[::5])

In [42]:
X = df.drop("y", axis=1).values
y = df.y.values

In [43]:
def df_to_X_y_seq(df, seq_len):
    X = []
    y = []

    for start, end in zip(np.arange(0, df.shape[0] - seq_len, seq_len), np.arange(seq_len, df.shape[0], seq_len)):
        X.append(df.iloc[start:end,:-1].values)
        y.append(df.iloc[start:end,-1].mode()[0])
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [46]:
X, y = df_to_X_y_seq(df, 800)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [47]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

ValueError: Found array with dim 3. Estimator expected <= 2.